In [4]:
import pandas as pd
import esig.tosig as ts
import numpy as np

In [5]:
file_path = "./datacsv/processed_battery_discharge_data.xlsx"
xls = pd.ExcelFile(file_path, engine='openpyxl')

In [6]:
# Set the signature truncation depth
signature_level = 3
result_data = []

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    if 'cycle' in df.columns and 'time' in df.columns and 'voltage' in df.columns and 'capacity' in df.columns:
        for cycle, group in df.groupby('cycle'):
            path = group[['time', 'voltage']].to_numpy()
            if len(path) < 2:
                continue
            try:
                sig = ts.stream2sig(path, signature_level)
            except:
                continue
            cap = group['capacity'].iloc[0]
            last_time = group['time'].iloc[-1] 
            label = int(cap >= 1.4)
            record = {
                'sheet_cycle': f'{sheet_name}_{cycle}',
                'capacity': cap,
                'event': label,
                'time': last_time  # 添加这一列
            }
            for idx, val in enumerate(sig):
                record[f's_{idx}'] = val
            result_data.append(record)

df_result = pd.DataFrame(result_data)
df_result.to_csv("./datacsv/battery_signature_features.csv", index=False)

In [9]:
# Corrected file path
file_path = './datacsv/battery_signature_features.csv'  # Make sure this path is correct

# Load the CSV file
df = pd.read_csv(file_path)

# Show the first 5 rows
print("First 5 rows of the CSV file:")
print(df.head())

First 5 rows of the CSV file:
         sheet_cycle  capacity  event      time  s_0       s_1    s_2  \
0  B0005_discharge_1  1.856487      1  3366.781  1.0  3350.000 -4.206   
1  B0005_discharge_2  1.846327      1  3368.688  1.0  3352.016 -4.204   
2  B0005_discharge_3  1.835349      1  3329.281  1.0  3312.484 -4.203   
3  B0005_discharge_4  1.835263      1  3329.500  1.0  3312.734 -4.203   
4  B0005_discharge_5  1.834646      1  3327.531  1.0  3310.812 -4.203   

            s_3          s_4          s_5       s_6           s_7  \
0  5.611250e+06 -8810.398990 -5279.701011  8.845218  6.265896e+09   
1  5.618006e+06 -8746.906560 -5344.968704  8.836808  6.277215e+09   
2  5.486275e+06 -8695.656696 -5226.713556  8.832605  6.057733e+09   
3  5.487103e+06 -8688.779905 -5234.641097  8.832605  6.059105e+09   
4  5.480738e+06 -8684.504525 -5230.838311  8.832605  6.048564e+09   

            s_8           s_9          s_10          s_11          s_12  \
0 -1.407010e+07 -1.374633e+06  11695.3006